In [18]:
import os
import boto3
import re
import json
from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

bucket='xgboosthoster' # put your s3 bucket name here, and create s3 bucket
prefix = 'sagemaker/dgaprediction'
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket)

***Cell 122 to 131 are useless since we tramsfer our model online***

In [122]:

import pandas as pd
import numpy as np
domain_df = pd.read_csv("initial_data.csv")
domain_df['new_domain'] = 'www.'+ domain_df.domain
domain_df.head(10)

,domain,domain_type,new_domain
0,wisuolycossttqrj.com,dga,www.wisuolycossttqrj.com
1,wi-wamss.org,benign,www.wi-wamss.org
2,qcxfurnkbqidxxcl.biz,dga,www.qcxfurnkbqidxxcl.biz
3,192-168-1-1-admin.ru,benign,www.192-168-1-1-admin.ru
4,dblsiobnkjxomkmh.ru,dga,www.dblsiobnkjxomkmh.ru
5,su5aksay.ru,benign,www.su5aksay.ru
6,sisalex.edu.eg,benign,www.sisalex.edu.eg
7,vloiolycossttqrj.cn,dga,www.vloiolycossttqrj.cn
8,bmabazar.com,benign,www.bmabazar.com
9,msyxwioqtcpbbypd.info,dga,www.msyxwioqtcpbbypd.info


In [123]:
def transform(val):
    vowel = ('a','e','i','o','u')
    new = ''
    for char in val:
        if char in vowel:
            new += 'v'
        elif char.isalpha() and char not in vowel:
            new += 'c'
        elif char.isdigit():
            new += 'n'
        else:
            new += 's'
    return new
domain_df['abbreviation']= domain_df['new_domain'].apply(transform)
domain_df['domain_type']= domain_df['domain_type'].apply(lambda x: 1 if x == 'benign' else 0)
domain_df.head(10)

,domain,domain_type,new_domain,abbreviation
0,wisuolycossttqrj.com,0,www.wisuolycossttqrj.com,cccscvcvvcccvcccccccscvc
1,wi-wamss.org,1,www.wi-wamss.org,cccscvscvcccsvcc
2,qcxfurnkbqidxxcl.biz,0,www.qcxfurnkbqidxxcl.biz,cccsccccvcccccvcccccscvc
3,192-168-1-1-admin.ru,1,www.192-168-1-1-admin.ru,cccsnnnsnnnsnsnsvccvcscv
4,dblsiobnkjxomkmh.ru,0,www.dblsiobnkjxomkmh.ru,cccsccccvvcccccvccccscv
5,su5aksay.ru,1,www.su5aksay.ru,cccscvnvccvcscv
6,sisalex.edu.eg,1,www.sisalex.edu.eg,cccscvcvcvcsvcvsvc
7,vloiolycossttqrj.cn,0,www.vloiolycossttqrj.cn,cccsccvvvcccvcccccccscc
8,bmabazar.com,1,www.bmabazar.com,cccsccvcvcvcscvc
9,msyxwioqtcpbbypd.info,0,www.msyxwioqtcpbbypd.info,cccscccccvvcccccccccsvccv


In [124]:
from nltk.util import ngrams
def extract_ngrams(row, num):
    n_grams = ngrams(row, num)
    return [ ''.join(grams) for grams in n_grams]
for i in range(1,4):
    domain_df['{}gram'.format(i)]= domain_df['abbreviation'].apply(extract_ngrams, args=(i,))
domain_df.head()

,domain,domain_type,new_domain,abbreviation,1gram,2gram,3gram
0,wisuolycossttqrj.com,0,www.wisuolycossttqrj.com,cccscvcvvcccvcccccccscvc,"[c, c, c, s, c, v, c, v, v, c, c, c, v, c, c, ...","[cc, cc, cs, sc, cv, vc, cv, vv, vc, cc, cc, c...","[ccc, ccs, csc, scv, cvc, vcv, cvv, vvc, vcc, ..."
1,wi-wamss.org,1,www.wi-wamss.org,cccscvscvcccsvcc,"[c, c, c, s, c, v, s, c, v, c, c, c, s, v, c, c]","[cc, cc, cs, sc, cv, vs, sc, cv, vc, cc, cc, c...","[ccc, ccs, csc, scv, cvs, vsc, scv, cvc, vcc, ..."
2,qcxfurnkbqidxxcl.biz,0,www.qcxfurnkbqidxxcl.biz,cccsccccvcccccvcccccscvc,"[c, c, c, s, c, c, c, c, v, c, c, c, c, c, v, ...","[cc, cc, cs, sc, cc, cc, cc, cv, vc, cc, cc, c...","[ccc, ccs, csc, scc, ccc, ccc, ccv, cvc, vcc, ..."
3,192-168-1-1-admin.ru,1,www.192-168-1-1-admin.ru,cccsnnnsnnnsnsnsvccvcscv,"[c, c, c, s, n, n, n, s, n, n, n, s, n, s, n, ...","[cc, cc, cs, sn, nn, nn, ns, sn, nn, nn, ns, s...","[ccc, ccs, csn, snn, nnn, nns, nsn, snn, nnn, ..."
4,dblsiobnkjxomkmh.ru,0,www.dblsiobnkjxomkmh.ru,cccsccccvvcccccvccccscv,"[c, c, c, s, c, c, c, c, v, v, c, c, c, c, c, ...","[cc, cc, cs, sc, cc, cc, cc, cv, vv, vc, cc, c...","[ccc, ccs, csc, scc, ccc, ccc, ccv, cvv, vvc, ..."


In [125]:
from collections import Counter
def generate_combinations(n, comb):
    if len(comb) == n:
        res.add(''.join(comb))
        return
    for char in ['c','v','s','n']:
        generate_combinations(n,comb+[char])
def bag_of_words_mapper(row, col):
    ngramlist = list(row)
    c = Counter(ngramlist)
    return c[col]
count = 1
for i in range(1,4):
    res = set()
    generate_combinations(i,[])
    for col in res:
        domain_df[col] =  domain_df['{}gram'.format(i)].apply(bag_of_words_mapper, args=(col,))
    df_ = domain_df[list(res)]
    domain_df['{}gram mean'.format(i)] = df_.mean(axis=1)
    domain_df['{}gram variance'.format(i)] = df_.var(axis=1)
    domain_df['{}gram std'.format(i)] = df_.std(axis=1)    

In [126]:
domain_df['F15'] = domain_df['new_domain'].apply(lambda x : len(set(x)))
domain_df['F16'] = domain_df['new_domain'].apply(lambda x : x.count('n')/len(x))
domain_df['F17'] = domain_df['new_domain'].apply(lambda x : x.count('c')/len(x))
domain_df['F18'] = domain_df['new_domain'].apply(lambda x : x.count('c')/x.count('v') if x.count('v') != 0 else x.count('c'))

In [127]:
X = domain_df.drop(['domain','1gram','2gram','3gram','abbreviation','new_domain'], axis=1)
y = X.pop('domain_type')

In [128]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=42)

In [129]:
from xgboost import XGBClassifier
model=XGBClassifier(objective='binary:logistic')
model.fit(X_train, y_train)
y_predict =model.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.9965
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2022
           1       1.00      0.99      1.00      1978

   micro avg       1.00      1.00      1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



col_0,0,1
row_0,,
0,2022,0
1,14,1964


In [130]:
model_file_name = "locally-fixed-xgboost-model"
model.save_model(model_file_name)

In [131]:
!tar czvf explicit_feature_model.tar.gz $model_file_name

locally-fixed-xgboost-model


In [19]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(region, 'xgboost', '0.90-1')

In [20]:
print(container)

746614075791.dkr.ecr.us-west-1.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3


In [133]:
### upload locally trained model (useless)
model_file_name = "locally-fixed-xgboost-model"
fObj = open("explicit_feature_model.tar.gz", 'rb')
key= os.path.join(prefix, model_file_name, 'explicit_feature_model.tar.gz')
boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(fObj)

In [21]:
import boto3
from time import gmtime, strftime

job_name = 'xgboost-dgaprediction-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Training job", job_name)

#Ensure that the training and validation data folders generated above are reflected in the "InputDataConfig" parameter below.

create_training_params = \
{
    "AlgorithmSpecification": {
        "TrainingImage": container,
        "TrainingInputMode": "File"
    },
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": bucket_path + "/" + prefix + "/locally-fixed-xgboost-model"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.m5.large",
        "VolumeSizeInGB": 1
    },
    "TrainingJobName": job_name,
    "HyperParameters": {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "silent":"0",
         "objective":"binary:logistic",
        "num_round":"50"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 3600
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": bucket_path + "/" + prefix + '/train',
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "libsvm",
            "CompressionType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": bucket_path + "/" + prefix + '/valid',
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "libsvm",
            "CompressionType": "None"
        }
    ]
}


client = boto3.client('sagemaker', region_name=region)
client.create_training_job(**create_training_params)
import time

status = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print(status)
while status !='Completed' and status!='Failed':
    time.sleep(60)
    status = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
    print(status)

Training job xgboost-dgaprediction-2020-05-10-01-36-21
InProgress
InProgress
InProgress
InProgress
Completed


In [22]:
from time import gmtime, strftime
model_name = "locally-fixed-xgboost-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

info = client.describe_training_job(TrainingJobName=job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

primary_container = {
    'Image': container,
    'ModelDataUrl': model_data,
}

create_model_response2 = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response2['ModelArn'])

https://s3-us-west-1.amazonaws.com/xgboosthoster/sagemaker/dgaprediction/locally-fixed-xgboost-model/xgboost-dgaprediction-2020-05-10-01-36-21/output/model.tar.gz
arn:aws:sagemaker:us-west-1:126529255912:model/locally-fixed-xgboost-model2020-05-10-01-40-32


In [23]:
from time import gmtime, strftime

endpoint_config_name = 'XGBoostEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.t2.medium',
        'InitialInstanceCount':1,
        'InitialVariantWeight':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

XGBoostEndpointConfig-2020-05-10-01-40-37
Endpoint Config Arn: arn:aws:sagemaker:us-west-1:126529255912:endpoint-config/xgboostendpointconfig-2020-05-10-01-40-37


In [24]:
import time

endpoint_name = 'XGBoostEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

XGBoostEndpoint-2020-05-10-01-40-41
arn:aws:sagemaker:us-west-1:126529255912:endpoint/xgboostendpoint-2020-05-10-01-40-41
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-1:126529255912:endpoint/xgboostendpoint-2020-05-10-01-40-41
Status: InService
